In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [2]:
df = pd.read_csv('data-week-3.csv')

df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')
df.totalcharges = df.totalcharges.fillna(0)

df.churn = (df.churn == 'yes').astype(int)

In [3]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [4]:
numerical = ['tenure', 'monthlycharges', 'totalcharges']

categorical = [
    'gender', 
    'seniorcitizen', 
    'partner', 
    'dependents',
    'phoneservice', 
    'multiplelines', 
    'internetservice',
    'onlinesecurity', 
    'onlinebackup', 
    'deviceprotection', 
    'techsupport',
    'streamingtv', 
    'streamingmovies', 
    'contract', 
    'paperlessbilling',
    'paymentmethod'
]

In [8]:
def train(df_train, y_train, C=1.0):
    dicts = df_train[categorical + numerical].to_dict(orient='records')
    
    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)
    
    model = LogisticRegression(C=C, max_iter=100000)
    model.fit(X_train, y_train)
    
    return dv, model

In [5]:
def predict(df, dv, model):
    dicts = df[categorical + numerical].to_dict(orient='records')
    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:, 1]
    
    return y_pred

In [6]:
C = 1.0
n_splits = 5

In [9]:
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

scores = []

for train_idx, val_idx in kfold.split(df_full_train):
    df_train_cv = df_full_train.iloc[train_idx]
    df_val_cv = df_full_train.iloc[val_idx]

    y_train = df_train_cv.churn.values
    y_val = df_val_cv.churn.values

    dv, model = train(df_train_cv, y_train, C=C)
    y_pred = predict(df_val_cv, dv, model)
        
    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)

print(f"C={C}: {np.mean(scores):.4f} +/- {np.std(scores):.4f}")

C=1.0: 0.8418 +/- 0.0069


In [10]:
scores

[0.844384611157774,
 0.8449812110978678,
 0.833257074051776,
 0.8347649005563728,
 0.8516480499513664]

In [12]:
dv, model = train(df_full_train, df_full_train.churn.values, C=1.0)
y_pred = predict(df_test, dv, model)

y_test = df_test.churn.values
      
auc = roc_auc_score(y_test, y_pred)
auc

0.858346333430834

### **Save the Model**

In [14]:
import pickle

In [15]:
# create an output file to save the model to
output_file = f"model_C={C}.bin"
output_file

'model_C=1.0.bin'

In [16]:
with open(output_file, "wb") as f_out:
    pickle.dump((dv, model), f_out)

### **Load the model**

In [1]:
import pickle

In [2]:
input_file = 'model_C=1.0.bin'

In [3]:
with open(input_file, 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [4]:
dv, model

(DictVectorizer(sparse=False), LogisticRegression(max_iter=100000))

In [5]:
customer = {
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'yes',
    'dependents': 'yes',
    'phoneservice': 'yes',
    'multiplelines': 'yes',
    'internetservice': 'fiber_optic',
    'onlinesecurity': 'yes',
    'onlinebackup': 'no',
    'deviceprotection': 'yes',
    'techsupport': 'no',
    'streamingtv': 'yes',
    'streamingmovies': 'yes',
    'contract': 'month-to-month',
    'paperlessbilling': 'yes',
    'paymentmethod': 'electronic_check',
    'tenure': 17,
    'monthlycharges': 104.2,
    'totalcharges': 1743.5
}

In [7]:
X = dv.transform([customer])

In [9]:
model.predict_proba(X)[0, 1]

np.float64(0.6382996215390251)